In [11]:
import numpy as np
import pandas as pd
import streamlit as st
import plotly.express as px
import re

In [12]:
cols = list(pd.read_excel("catan_data.xlsx").iloc[0,:])

data = pd.read_excel("catan_data.xlsx").iloc[1:,:]

data.columns = cols

data["game_id"] = data["season"]*100 + data["game"]

C:\Users\maxfu\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\maxfu\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [13]:
data

,season,Session,game,loc,geoloc,player,score,special,place,position,...,prox_port_gen,t_sum_wood,t_sum_clay,t_sum_sheep,t_sum_grain,t_sum_ore,t_sum_paper,t_sum_coin,t_sum_fabric,game_id
1,2022,2022-03-19 00:00:00,1,Husum (Memeler Str.),"54.491864, 9.030382",PF,7,NaN,2,first,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202201
2,2022,2022-03-19 00:00:00,1,Husum (Memeler Str.),"54.491864, 9.030382",MF,14,NaN,1,second,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202201
3,2022,2022-03-19 00:00:00,1,Husum (Memeler Str.),"54.491864, 9.030382",JHC,5,NaN,3,third,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202201
4,2022,2022-03-19 00:00:00,2,Husum (Memeler Str.),"54.491864, 9.030382",JHC,7,NaN,2,first,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202202
5,2022,2022-03-19 00:00:00,2,Husum (Memeler Str.),"54.491864, 9.030382",MF,13,NaN,1,second,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287,2024,2024-12-20 00:00:00,44,Husum (Memeler Str.),"54.491864, 9.030382",MF,5,1,3,second,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202444
288,2024,2024-12-20 00:00:00,44,Husum (Memeler Str.),"54.491864, 9.030382",PF,13,5,1,third,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202444
289,2024,2024-12-20 00:00:00,45,Husum (Memeler Str.),"54.491864, 9.030382",JHC,13,5,1,first,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202445
290,2024,2024-12-20 00:00:00,45,Husum (Memeler Str.),"54.491864, 9.030382",MF,10,2,2,second,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202445


In [14]:
points_cum = data[["game_id","Session","season","game","loc","player","place"]].copy()

In [15]:
points_cum["points"] = points_cum["place"].map({1:2,2:1,3:0})
points_cum["points_cum"] = np.nan
points_cum["points_cum_ytd"] = np.nan

In [16]:
points_cum_jhc = points_cum.loc[points_cum["player"]=="JHC"]
points_cum_pf = points_cum.loc[points_cum["player"]=="PF"]
points_cum_mf = points_cum.loc[points_cum["player"]=="MF"]

In [17]:
for dude in [points_cum_jhc,points_cum_mf,points_cum_pf]:
    for i in dude.index:
        
        cutoff = dude.loc[dude.index==i]["game_id"].iloc[0]
        #kumulierte gesamtpunkte setzen
        dude.loc[dude.index==i,"points_cum"] = dude.loc[dude["game_id"]<=cutoff]["points"].sum()
        # kumulierte punkte seit jahresbeginn
        dude.loc[dude.index==i,"points_cum_ytd"] = dude.loc[(dude["game_id"]<=cutoff) & (dude["season"]==int(str(cutoff)[:4]))]["points"].sum()
        

In [33]:
points_cum_jhc

,game_id,Session,season,game,loc,player,place,points,points_cum,points_cum_ytd
3,202201,2022-03-19 00:00:00,2022,1,Husum (Memeler Str.),JHC,3,0,0.0,0.0
4,202202,2022-03-19 00:00:00,2022,2,Husum (Memeler Str.),JHC,2,1,1.0,1.0
9,202203,2022-03-19 00:00:00,2022,3,Husum (Memeler Str.),JHC,1,2,3.0,3.0
10,202204,2022-03-19 00:00:00,2022,4,Husum (Memeler Str.),JHC,3,0,3.0,3.0
13,202205,2022-03-19 00:00:00,2022,5,Husum (Memeler Str.),JHC,3,0,3.0,3.0
...,...,...,...,...,...,...,...,...,...,...
278,202441,2024-12-20 00:00:00,2024,41,Husum (Memeler Str.),JHC,3,0,97.0,40.0
280,202442,2024-12-20 00:00:00,2024,42,Husum (Memeler Str.),JHC,1,2,99.0,42.0
283,202443,2024-12-20 00:00:00,2024,43,Husum (Memeler Str.),JHC,2,1,100.0,43.0
286,202444,2024-12-20 00:00:00,2024,44,Husum (Memeler Str.),JHC,2,1,101.0,44.0


In [87]:
# Calculate mean scores for each player at each location
mean_scores = data.groupby(['loc', 'player'])['score'].mean().unstack().fillna(0)
location_count = data['loc'].value_counts() / 3


# Create a hover text column
hover_text_scores = mean_scores.apply(lambda row: '\n'.join([f"{player}: {score:.2f}" for player, score in row.items()]), axis=1)
hover_text = "Punktedurchschnitt: " + hover_text_scores + '\n' + 'Anzahl Spiele: ' + location_count.reindex(hover_text_scores.index).astype(float).apply(lambda x: f"{x:.1f}")
# Merge this with your original data
data = data.merge(hover_text.rename('hover_info'), on='loc', how='left')
# Ensure unique entries for locations to avoid plot duplication
data_unique = data.drop_duplicates(subset=['loc'])

In [88]:
hover_text

loc
Hamburg (Kegelhofstr.)    Punktedurchschnitt: JHC: 10.53\nMF: 8.37\nPF: ...
Husum (Memeler Str.)      Punktedurchschnitt: JHC: 9.88\nMF: 8.12\nPF: 9...
Köln (Nikolausstr.)       Punktedurchschnitt: JHC: 8.25\nMF: 7.38\nPF: 9...
Online                    Punktedurchschnitt: JHC: 11.33\nMF: 8.33\nPF: ...
dtype: object